In [ ]:
import os
import io
from googleapiclient.http import MediaIoBaseDownload
from Google import Create_Service

CLIENT_SECRET_FILE = 'client_secret_GoogleCloudDemo.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE,API_NAME,API_VERSION,SCOPES)

q = "'1YRE-qSlUtKpwq3yGp0rqUcYtRZxIF64s' in parents and mimeType='application/vnd.google-apps.folder'"

response1 = service\
.files()\
.list(q=q,\
      spaces='drive',\
      corpora='user')\
.execute()

In [ ]:
id_list=[]
name_list=[]
for file in response1.get('files',[]):
    q="name contains 'co_newres' and '%s' in parents and \
    (mimeType='application/vnd.ms-excel' or \
    mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')" % (file['id'])
    response = service\
    .files()\
    .list(q=q,\
          spaces='drive',\
          corpora='user')\
    .execute()
    
    for element in response['files']:
        id_list.append(element['id'])
        name_list.append(element['name'])


In [ ]:
for file_id, file_name in zip(id_list,name_list):
    request = service.files().get_media(fileId=file_id)
#     request = service.files().export_media(fileId=file_id,mimeType='text/csv') does not work since the files were not
#     originally google workbooks
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fd=fh,request=request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print('Download Progress {0}'.format(status.progress()*100))
    fh.seek(0)
    with open(os.path.join(os.getcwd(),file_name),'wb') as f:
        f.write(fh.read())
        f.close()